In [ ]:
%run ANNOTATIONS.ipynb

In [ ]:
%run PATTERN_OBSERVER.ipynb

In [ ]:
from ipywidgets import VBox,HBox,Text,DatePicker,Button,Layout,Dropdown
from ipywidgets import Color # pretty color for apply button
from pathlib import Path # for parsing directory contents

In [ ]:
'''
backtest location
date range
skip dates - TODO
return compute type (cash,future,both)
'''
class SETTINGS(ISubject):
    ####################################
    # static
    ####################################
    _default_backtest_path=r'C:\Users\ahkar\OneDrive\Documents\Data\B3'
    
    _return_types=[
        'Cash + Future',
        'Cash',
        'Future',
    ]
    
    '''each entry of SETTINGS._option_names needs to be a widget and also instantiable'''
    _option_names=[
        'backtest_path',
        'date_from',
        'date_to',
        'return_type',
    ]

    ####################################
    # constructor
    ####################################
    def __init__(self,
        name='SETTINGS',
        width='25%'
        ):
        self._DEBUG=True
        self._width=width
        
        self.name=name
        self.widget=self._make()
        
        self.applied_settings=self._current_settings # update checkpoint
        self._update_reference() # update reference
        self._observers=set() # for observer pattern
        
    ####################################
    # observer pattern
    ####################################
    # subject
    def attach(self,observer : IObserver) -> None :
        print('OBSERVER PATTERN',':',observer.name,'OBSERVES',self.name)
        self._observers.add(observer)
        
    def detach(self,observer : IObserver) -> None :
        print('OBSERVER PATTERN',':',observer.name,'STOPS OBSERVING',self.name)
        self._observers.remove(observer)
        
    def notify(self,info) -> None :
        print('OBSERVER PATTERN',':',self.name,'NOTIFIES',len(self._observers),'OBSERVERS')
        
        # print receivers
        for observer in self._observers:
            print('OBSERVER PATTERN',':',self.name,'NOTIFIES',observer.name)
            observer.react(self.name,info)
            
    ####################################
    # buttons
    ####################################
    def _button_inactive(self,button : Button) -> None :
        button.style.button_color=Color(None).name
        button.disabled=True
        
    def _button_active(self,button : Button,color : str ='pink') -> None :
        button.style.button_color=color
        button.disabled=False
        
    def _update_button_state(self) -> None :
        if self._DEBUG:
            print('_update_button_state','called')
            
        if self.applied_settings==self._current_settings:
            # if nothing changed
            self._button_inactive(self._control_widgets[0])
            self._button_inactive(self._control_widgets[1])
        else:
            # if something changed
            self._button_active(self._control_widgets[0],color='lightgreen')
            self._button_active(self._control_widgets[1],color='pink')

    def _click_apply(self,change) -> None :
        print(self.name,':','CLICKED',change.description)
        self.applied_settings=self._current_settings # update checkpoint
        self._update_button_state() # update button state
        self._update_reference() # update reference
        self.notify(self.reference) # notify observers

    '''reset to last applied values'''
    def _click_cancel(self,change) -> None :
        print(self.name,':','CLICKED',change.description)
        for widget,value in zip(self._option_widgets,self.applied_settings): widget.value=value

    def _click_reset(self,change) -> None :
        print(self.name,':','CLICKED',change.description)
        self._option_widgets[1].value=None
        self._option_widgets[2].value=None
        self._option_widgets[3].value=SETTINGS._return_types[0]

    def _click_refresh(self,change) -> None :
        self._click_cancel(change) # forget current settings
        self.notify(self.reference) # notify observers with current state of self.reference

    def _click_reload(self,change) -> None :
        self._click_cancel(change) # forget current settings
        self.notify({'force_reload':True}) # notify observers with `force_reload` message
        
    def _onchange_backtest_path(self,change) -> None :
        print(self.name,':','CHANGED',change.owner.description,change.new)
        self._update_button_state() # update button state
        
    def _onchange_date_from(self,change) -> None :
        print(self.name,':','CHANGED',change.owner.description,change.new)
        self._update_button_state() # update button state

    def _onchange_date_to(self,change) -> None :
        print(self.name,':','CHANGED',change.owner.description,change.new)
        self._update_button_state() # update button state
        
    def _onchange_return_type(self,change) -> None :
        print(self.name,':','CHANGED',change.owner.description,change.new)
        self._update_button_state() # update button state

    ####################################
    # GUI
    ####################################
    def _make(self) -> VBox :
        # controls
        controls=HBox(self._make_controls())
        
        # options
        options=VBox(
            children=self._make_options(),
            layout=Layout(height='300px')
        )
        
        # return
        return VBox(
            children=[controls,options],
            layout=Layout(border='1px solid black',width=self._width,overflow_x='scroll')
        )
    
    '''checkbox controls'''
    def _make_controls(self) -> BUTTONS :
        button_apply=Button(description='apply',disabled=True)
        button_apply.on_click(self._click_apply)

        button_cancel=Button(description='cancel',disabled=True)
        button_cancel.on_click(self._click_cancel)

        button_reset=Button(description='reset')
        button_reset.on_click(self._click_reset)

        button_refresh=Button(description='refresh')
        button_refresh.on_click(self._click_refresh)

        button_reload=Button(description='reload')
        button_reload.on_click(self._click_reload)

        return [
            button_apply,
            button_cancel,
            button_reset,
            button_refresh,
            button_reload,
        ]

    '''make list of options'''
    def _make_options(self) -> WIDGETS:
        # backtest_path
        backtest_path=Text(
            description='path',
            value=SETTINGS._default_backtest_path,
            placeholder='what is a placeholder?',
            disabled=False,
            #continuous_update=False,
        )
        backtest_path.observe(self._onchange_backtest_path,names='value')
        
        # date_from
        date_from=DatePicker(description='date from')
        date_from.observe(self._onchange_date_from,names='value')
        
        # date_to
        date_to=DatePicker(description='date to')
        date_to.observe(self._onchange_date_to,names='value')
            
        # return_type
        return_type=Dropdown(description='return type',options=SETTINGS._return_types)
        return_type.observe(self._onchange_return_type,names='value')
        
        #instantiate and return list of widgets / options
        return list(map(eval,SETTINGS._option_names))

    @property
    def _control_widgets(self) -> BUTTONS :
        return self.widget.children[0].children
    
    @property
    def _option_widgets(self) -> WIDGETS :
        return self.widget.children[1].children
    
    @property
    def _current_settings(self) -> List :
        return [x.value for x in self._option_widgets]
    
    '''get applied setting by name'''
    def _applied_setting_by_name(self,name : str):
        return self.applied_settings[SETTINGS._option_names.index(name)]

    '''return list of contents in backtest_path'''
    def _refresh_all_backtests(self) -> ID_PATHS :
        p=self._applied_setting_by_name('backtest_path')
        return {x.stem:x for x in Path(p).iterdir()}
        
    '''return list of books in backtest_path'''
    def _refresh_all_books(self) -> ID_PATHS :
        # TODO update this
        return {x:x for x in ['Trading','Quote','MM2','Hedge','Hit']}

    '''
    # TODO this assumes settings are correct, backtest_path can be wrong!
    '''
    def _update_reference(self) -> None:
        self.reference={
            'all_backtests':self._refresh_all_backtests(),                              # ID_PATHS
            'all_books':self._refresh_all_books(),                                      # ID_PATHS
            'backtest_path':self._applied_setting_by_name('backtest_path'),             # str
            'date_from':self._applied_setting_by_name('date_from'),                     # DateTime
            'date_to':self._applied_setting_by_name('date_to'),                         # DateTime
            'return_type':self._applied_setting_by_name('return_type'),                 # str
        }
                                                                                        
        # show internals
        self._logging()
        
    def _logging(self) -> None:
        for k,v in self.reference.items():
            print(self.name,':','REFERENCE',':',k,':',v)

In [ ]:
# example
SETTINGS().widget